In [53]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.options.display.float_format = '{:,.2f}'.format

In [54]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [55]:
def search_pay_func(mes: int) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    

    param_dia_ini = datetime(2024, mes, 1)
    param_dia_fin = datetime(2024, mes + 1, 1) - timedelta(days= 1)

    search_pay_acc = [
        "&", "&",
            ("partner_type", "=", "customer"),
            ("is_internal_transfer", "=", False),
        "&",
            ("date", ">=", param_dia_ini.strftime('%Y-%m-%d')),
            ("date", "<=", param_dia_fin.strftime('%Y-%m-%d')),
        ]


    param_dia_ini_pos = param_dia_ini + timedelta(hours=7)
    param_dia_fin_pos = param_dia_fin + timedelta(hours=31) - timedelta(seconds= 1)
    
    search_pay_pos = [
        "&",
            ("payment_date", ">=", param_dia_ini_pos.strftime('%Y-%m-%d %H:%M:%S')),
            ("payment_date", "<=", param_dia_fin_pos.strftime('%Y-%m-%d %H:%M:%S')),
        ]
    

    return search_pay_acc, search_pay_pos

In [56]:
def api_call_pay_acc_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    search_pay_acc = search_pay[0]


    pay_acc_fields = [
                    'name',
                    'date',
                    'partner_id',
                    'amount_total',
                    'ref',
                    'reconciled_invoice_ids',
                    'pos_session_id'
                    ]

    pay_acc_ids = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'search', [search_pay_acc])
    pay_acc_json = models.execute_kw(api_db, uid, api_clave, 'account.payment', 'read', [pay_acc_ids], {'fields': pay_acc_fields})

    
    return pay_acc_json

In [57]:
def pay_acc_df_func(pay_acc_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_acc = []
    data_fact_ids = []

    for pay in pay_acc_json:
        if not pay['pos_session_id']:
            new = {}
            new['id'] = pay['id']
            new['name'] = pay['name']
            new['date'] = pay['date']
            new['partner_id'] = pay['partner_id']
            new['amount_total'] = pay['amount_total']
            new['ref'] = pay['ref'] if pay['ref'] else pd.NA
            new['pay_fact_docs'] = pay['reconciled_invoice_ids'] if pay['reconciled_invoice_ids'] else pd.NA
            
            data_fact_ids += pay['reconciled_invoice_ids']

            data_pay_acc.append(new)

    pay_acc_df = pd.DataFrame(data_pay_acc)


    pay_acc_df['date'] = pd.to_datetime(pay_acc_df['date'], format='%Y-%m-%d')
    
    return pay_acc_df, data_fact_ids

In [58]:
def api_call_pay_pos_func(api_params: dict, search_pay: list[str] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    search_pay_pos = search_pay[1]


    pay_pos_fields = [
                    'pos_order_id',
                    'payment_date',
                    'amount',
                    'payment_method_id'
                    ]

    pay_pos_ids = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'search', [search_pay_pos])
    pay_pos_json = models.execute_kw(api_db, uid, api_clave, 'pos.payment', 'read', [pay_pos_ids], {'fields': pay_pos_fields})
    

    return pay_pos_json

In [59]:
def pay_pos_df_func(pay_pos_json: list[dict]) -> list[pd.DataFrame, int]:
    
    data_pay_pos = []
    data_pos_ids = set()

    for pay in pay_pos_json:
        if pay['payment_method_id'][0] != 6:
            new = {}
            new['pay_pos_id'] = pay['id']
            new['pos_order_id'] = pay['pos_order_id'][0]
            new['pos_order_name'] = pay['pos_order_id'][1]
            new['date'] = pay['payment_date']
            new['amount'] = pay['amount']
        
            data_pos_ids.add(pay['pos_order_id'][0])
            data_pay_pos.append(new)

    pay_pos_df1 = pd.DataFrame(data_pay_pos)
    pay_pos_df = pay_pos_df1.groupby('pos_order_id').agg({'pay_pos_id':['first'], 'pos_order_name':['first'], 'date':['first'], 'amount':['sum']}).reset_index().iloc[:,[1,0,2,3,4]]
    pay_pos_df.columns = ['pay_pos_id', 'pos_order_id', 'pos_order_name', 'date', 'amount']

    pay_pos_df['date'] = pd.to_datetime(pay_pos_df['date'], format='%Y-%m-%d %H:%M:%S')
    
    return pay_pos_df, list(data_pos_ids)

In [60]:
def api_call_pos_order_func(api_params: dict, data_pos_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    pos_order_fields = [
                    'name',
                    'account_move',
                    'partner_id',
                    'session_id',
                    'amount_total'
                    ]

    pos_order_json = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'read', [data_pos_ids], {'fields': pos_order_fields})

    return pos_order_json

In [61]:
def pos_order_func(pos_order_json: list[dict], data_fact_ids: list[int]) -> list[list[int]]:
    
    data_pos_amount_cero = [] 
  
    for pos in pos_order_json:
        
        if pos['account_move']:
            data_fact_ids.append(pos['account_move'][0])

        if pos['amount_total'] == 0:
            data_pos_amount_cero.append(pos)


    pos_amount_cero_df = pd.DataFrame(data_pos_amount_cero)
    pos_amount_cero_df['session_id'] = pos_amount_cero_df['session_id'].str.get(1)
        
    
    return data_fact_ids, pos_amount_cero_df

In [62]:
def api_call_acccount_docs_func(api_params: dict, data_fact_ids: list[int] ) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    data_fact_fields = [
                    'name',
                    'partner_id',
                    'date',
                    'invoice_payments_widget',
                    'amount_total',
                    'amount_residual'
                    ]

    fact_doc_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [data_fact_ids], {'fields': data_fact_fields})

    return fact_doc_json

In [63]:
api_params = api_params_func()
search_pay = search_pay_func(4)

pay_acc_json = api_call_pay_acc_func(api_params, search_pay)
pay_acc_df, data_fact_ids2 = pay_acc_df_func(pay_acc_json)

pay_pos_json = api_call_pay_pos_func(api_params, search_pay)
pay_pos_df, data_pos_ids = pay_pos_df_func(pay_pos_json)

pos_order_json = api_call_pos_order_func(api_params, data_pos_ids)
data_fact_ids, pos_amount_cero_df = pos_order_func(pos_order_json, data_fact_ids2)

fact_doc_json = api_call_acccount_docs_func(api_params, data_fact_ids)

In [64]:
data_pay_fact = []

for fact in fact_doc_json:
    if fact['invoice_payments_widget']:
        for pay in fact['invoice_payments_widget']['content']:
             if 'Facturas' not in pay['journal_name'] and datetime.strptime(pay['date'], '%Y-%m-%d').month == 4:
                new = {}
                new['fac_doc_id'] = fact['id']
                new['fac_doc_name'] = fact['name']
                new['fac_doc_cliente'] = fact['partner_id'][1]
                new['fac_doc_date'] = fact['date']
                new['fac_doc_total'] = fact['amount_total']
                new['fac_doc_deuda'] = fact['amount_residual']
                new['pay_journal'] = pay['journal_name']
                new['pay_amount'] = pay['amount']
                new['pay_date'] = pay['date']
                new['pay_pos'] = pay['pos_payment_name'] if pay['pos_payment_name'] else pd.NA
    
                data_pay_fact.append(new)

pay_fact_df = pd.DataFrame(data_pay_fact)
pay_fact_df['fac_doc_date'] = pd.to_datetime(pay_fact_df['fac_doc_date'], format='%Y-%m-%d')
pay_fact_df['pay_date'] = pd.to_datetime(pay_fact_df['pay_date'], format='%Y-%m-%d')

In [136]:
pdv_cobranza_df = pos_amount_cero_df.merge(pay_pos_df, how='left', left_on='id', right_on='pos_order_id')
pdv_cobranza_df.head()

,id,name,account_move,partner_id,session_id,amount_total,pay_pos_id,pos_order_id,pos_order_name,date,amount
0,11510,PdV SJC/6527,False,"[13024, CLAUDIA ANTONIETA PADILLA CANALES]",POS/00169,0.00,16303,11510,PdV SJC/6527,2024-04-02 22:06:15,"2,000.00"
1,11873,PdV CSL/4890,False,"[16087, VIDAL COSTICH CORTES]",POS/00174,0.00,16817,11873,PdV CSL/4890,2024-04-05 16:05:28,751.79
2,12119,PdV CSL/5003,False,"[15408, GUADALUPE (SR.PINACHO)]",POS/00176,0.00,17177,12119,PdV CSL/5003,2024-04-06 22:23:47,"25,354.40"
3,12866,PdV CSL/5341,False,"[233, Carlos Humberto Briones Contreras]",POS/00186,0.00,18264,12866,PdV CSL/5341,2024-04-12 18:38:15,"1,365.86"
4,12867,PdV CSL/5342,False,"[16628, Gabriel Higuera Arce]",POS/00186,0.00,18266,12867,PdV CSL/5342,2024-04-12 18:39:28,402.60


In [134]:
acc_cobranza_pdv_df = pay_acc_df[(~pay_acc_df['ref'].isna()) & (pay_acc_df['ref'].str[:1] != 'F')]

In [138]:
for i in range(len(pdv_cobranza_df['id'])):
    mini_df = acc_cobranza_pdv_df.loc[(acc_cobranza_pdv_df['ref'].str.contains('PdV SJC/6527')) 
         | (acc_cobranza_pdv_df['ref'].str.contains('POS/00169')) 
         & (acc_cobranza_pdv_df['amount_total'] == 2000)
         & (acc_cobranza_pdv_df['partner_id'].str.get(0) == 13024)]

if not mini_df.empty:
    print(mini_df['id'].iloc[0])


1306


In [68]:
pay_fact_df[['pay_pos', 'pay_amount']].groupby('pay_pos').agg({'pay_amount': ['count', 'sum']})

pay_amount           
                                         count        sum
pay_pos                                                  
Efectivo A1                               1032 375,707.14
Efectivo A2                               1194 424,217.70
Tarjeta Crédito A1                         136 149,800.80
Tarjeta Crédito A2                         220 305,872.22
Tarjeta Débito A1                          458 435,937.31
Tarjeta Débito A2                          523 507,382.38
Transferencia Banco Sant92000702524        144 792,272.16
Transferencia Banco Scot23900002860          6   6,510.51

In [69]:
# pay_fact_df.loc[pay_fact_df['pay_pos'] == 'Efectivo A1'].to_excel('Efectivo_A1.xlsx')
pay_fact_df.loc[pay_fact_df['pay_pos'] == 'Efectivo A1']

,fac_doc_id,fac_doc_name,fac_doc_cliente,fac_doc_date,fac_doc_total,fac_doc_deuda,pay_journal,pay_amount,pay_date,pay_pos
755,52511,F1-CC/2024/04674,CRUZ DOMINGO CHAVEZ CAMACHO,2024-04-01,964.11,0.00,Punto De Venta A1,964.11,2024-04-01,Efectivo A1
757,52519,F1-CC/2024/04676,Mostrador,2024-04-01,35.35,0.00,Punto De Venta A1,35.35,2024-04-01,Efectivo A1
769,52567,F1-CC/2024/04679,Mostrador,2024-04-01,86.90,0.00,Punto De Venta A1,86.90,2024-04-01,Efectivo A1
775,52592,F1-CC/2024/04681,Mostrador,2024-04-01,364.07,0.00,Punto De Venta A1,364.07,2024-04-01,Efectivo A1
776,52595,F1-CC/2024/04682,Abiel Ceron Moreno,2024-04-01,"1,820.78",0.00,Punto De Venta A1,"1,820.78",2024-04-01,Efectivo A1
...,...,...,...,...,...,...,...,...,...,...
4437,72488,F1-CC/2024/06496,Emilio Costich Lopez,2024-04-30,169.43,0.00,Punto De Venta A1,169.43,2024-04-30,Efectivo A1
4441,72511,F1-CC/2024/06497,Emilio Costich Lopez,2024-04-30,134.85,0.00,Punto De Venta A1,134.85,2024-04-30,Efectivo A1
4445,72525,F1-CC/2024/06499,Emilio Costich Lopez,2024-04-30,164.28,0.00,Punto De Venta A1,164.28,2024-04-30,Efectivo A1
4446,72532,F1-CC/2024/06500,Emilio Costich Lopez,2024-04-30,289.21,0.00,Punto De Venta A1,289.21,2024-04-30,Efectivo A1


In [70]:
pay_fact_df[['pay_journal', 'pay_amount']].groupby('pay_journal').agg({'pay_amount': ['count', 'sum']})

pay_amount             
                           count          sum
pay_journal                                  
Banco Bana70103464895         11   116,869.17
Banco Sant65507231316        555 3,426,051.23
Banco Sant92000702524        122   334,665.52
Efectivo A1                   52    69,940.72
Efectivo A2                    5     4,745.57
Punto De Venta A1           1676 1,126,484.53
Punto De Venta A2           2037 1,871,215.69
Tarjeta Deb/Cré A1             1    21,935.60

In [71]:
pay_fact_df.agg({'pay_amount':['sum']})

,pay_amount
sum,"6,971,908.03"


## Pruebas